In [16]:
!pip install pandas geopandas python-dotenv requests


[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import os
import geopandas as gpd
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

base_folder = '地価公示属性編集済データ'
target_crs = 'EPSG:6668'
default_crs = 'EPSG:6668'


# シェープファイルを再帰的に検索する関数
def find_shapefiles(target_folder):
    shapefile_paths = []
    for root, dirs, files in os.walk(target_folder):
        for dir_name in dirs:
            if dir_name.startswith("L01") or dir_name.startswith("L02"):
                year_folder_path = os.path.join(root, dir_name)
                for sub_root, _, sub_files in os.walk(year_folder_path):
                    for file in sub_files:
                        if file.endswith('.shp'):
                            shapefile_path = os.path.join(sub_root, file)
                            shapefile_paths.append(shapefile_path)
    return shapefile_paths


# シェープファイルを並列処理で読み込む関数
def load_shapefile(shapefile_path):
    gdf = gpd.read_file(shapefile_path)
    if gdf.crs is None:
        gdf.set_crs(default_crs, inplace=True)
    if gdf.crs != target_crs:
        gdf = gdf.to_crs(target_crs)
    return gdf


# シェープファイルを読み込む関数（並列処理）
def load_shapefiles_parallel(shapefile_paths):
    shapefiles = []
    with ThreadPoolExecutor() as executor:
        results = executor.map(lambda p: load_shapefile(p), shapefile_paths)
        for result in results:
            shapefiles.append(result)
    return shapefiles


# 地価公示と地価調査のシェープファイルを検索して読み込む
koji_shapefile_paths = find_shapefiles(base_folder)
koji_shapefiles = load_shapefiles_parallel(koji_shapefile_paths)

# GeoDataFrameを結合して一つにする
gdf_koji_all = gpd.GeoDataFrame(pd.concat(koji_shapefiles, ignore_index=True))

# 結合結果を確認
print(gdf_koji_all.head())
print(gdf_koji_all.columns)
gdf_koji_all.to_csv('gdf_koji_all.csv', encoding="utf-8-sig", index=False)

  L01_001 L01_002 L01_003 L01_004 L01_005 L01_006 L01_007 L01_008 L01_009  \
0     000     001     000     001    2007   45700       1   false   false   
1     005     002     005     002    2007   72300       1   false   false   
2     000     002     000     002    2007   45000       1   false   false   
3     005     001     005     001    2007  135000       1   false   false   
4     000     003     000     003    2007   37500       1   false   false   

  L01_010  ... L01_135 L01_136 L01_137 L01_138 L01_139 L01_140 L01_141  \
0   false  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1   false  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2   false  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3   false  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4   false  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

  L01_142 L01_143 L01_144  
0     NaN     NaN     NaN  
1     NaN     NaN     NaN  
2     Na

In [18]:
# ここでgdf_koji_allは既にシェープファイルを結合したGeoDataFrameと仮定します
# 必要なカラムを持つデータフレームを抽出
columns_to_extract = ['year', 'price', 'area_code']

# 必要なカラムがすべて存在するか確認
existing_columns = [col for col in columns_to_extract if col in gdf_koji_all.columns]

# 必要なカラムを持つ新しいデータフレームを作成
extracted_df = gdf_koji_all[existing_columns]

# 結果を確認
print(extracted_df.head())
print(extracted_df.columns)

# データフレームを保存する場合
# extracted_df.to_csv('extracted_data.csv', index=False)
print(extracted_df['year'].value_counts().to_string())

   year   price area_code
0  2007   45700     47207
1  2007   72300     47207
2  2007   45000     47207
3  2007  135000     47207
4  2007   37500     47207
Index(['year', 'price', 'area_code'], dtype='object')
year
2007    30000
2008    29100
2009    28227
2010    27804
2011    26000
2024    25994
2019    25993
2021    25993
2020    25993
2022    25993
2023    25993
2017    25988
2018    25988
2012    25983
2013    25983
2016    25255
2015    23363
2014    23363


In [33]:
# 年度ごとの全国平均価格を計算
national_avg = extracted_df.groupby('year')['price'].mean().reset_index()
# 年次リターンの計算
national_avg['return'] = national_avg['price'].pct_change()

# 先頭行のリターンは計算できないため削除
national_avg = national_avg.dropna(subset=['return'])
print(national_avg.to_string())
national_avg

    year          price    return
1   2008  207159.301375  0.122356
2   2009  193554.755872 -0.065672
3   2010  176429.304525 -0.088479
4   2011  173823.028346 -0.014772
5   2012  170443.033406 -0.019445
6   2013  170770.612708  0.001922
7   2014  175010.528186  0.024828
8   2015  179981.496811  0.028404
9   2016  190666.582459  0.059368
10  2017  201411.248769  0.056353
11  2018  211460.159420  0.049892
12  2019  224177.323472  0.060140
13  2020  238070.937560  0.061976
14  2021  234067.730120 -0.016815
15  2022  235223.898934  0.004939
16  2023  242691.412688  0.031746
17  2024  255783.518004  0.053945


,year,price,return
1,2008,207159.301375,0.122356
2,2009,193554.755872,-0.065672
3,2010,176429.304525,-0.088479
4,2011,173823.028346,-0.014772
5,2012,170443.033406,-0.019445
6,2013,170770.612708,0.001922
7,2014,175010.528186,0.024828
8,2015,179981.496811,0.028404
9,2016,190666.582459,0.059368
10,2017,201411.248769,0.056353


In [20]:
import numpy as np

# 指標の計算関数
def calculate_metrics(group):
    if len(group) < 2:
        return pd.Series({
            'CAGR': np.nan,
            'Geometric Mean Return': np.nan,
            'Annualized Volatility': np.nan,
            'Sharpe Ratio': np.nan
        })
    
    cagr = ((group['price'].iloc[-1] / group['price'].iloc[0]) ** (1 / (group['year'].iloc[-1] - group['year'].iloc[0]))) - 1
    geometric_mean_return = (np.prod(1 + group['return']) ** (1 / group['return'].count())) - 1
    volatility = group['return'].std()
    sharpe_ratio = geometric_mean_return / volatility if volatility != 0 else np.nan
    return pd.Series({
        'CAGR': cagr,
        'Geometric Mean Return': geometric_mean_return,
        'Annualized Volatility': volatility,
        'Sharpe Ratio': sharpe_ratio
    })

# 全国の指標を計算
national_metrics = calculate_metrics(national_avg)

# 結果をフォーマットする関数
def format_metrics(metrics):
    formatted_metrics = {
        'CAGR': f"{metrics['CAGR'] * 100:.2f}%" if not np.isnan(metrics['CAGR']) else 'N/A',
        'Geometric Mean Return': f"{metrics['Geometric Mean Return'] * 100:.2f}%" if not np.isnan(metrics['Geometric Mean Return']) else 'N/A',
        'Annualized Volatility': f"{metrics['Annualized Volatility'] * 100:.2f}%" if not np.isnan(metrics['Annualized Volatility']) else 'N/A',
        'Sharpe Ratio': f"{metrics['Sharpe Ratio']:.3f}" if not np.isnan(metrics['Sharpe Ratio']) else 'N/A'
    }
    return pd.Series(formatted_metrics)

# 全国の指標を表示
formatted_national_metrics = format_metrics(national_metrics)
df_national_metrics = pd.DataFrame([formatted_national_metrics], index=["全国"])
df_national_metrics.index.name = "national"
df_national_metrics.to_csv("national_metrics.csv", encoding="utf-8-sig", index=True)
print("National Metrics:")
df_national_metrics

National Metrics:


,CAGR,Geometric Mean Return,Annualized Volatility,Sharpe Ratio
national,,,,
全国,1.33%,1.94%,5.17%,0.375


In [21]:
import requests
from dotenv import load_dotenv
import os

# .envファイルの内容を読み込む
load_dotenv()

# RESAS-APIのエンドポイントとAPIキー
pref_url  = "https://opendata.resas-portal.go.jp/api/v1/prefectures"
city_url = "https://opendata.resas-portal.go.jp/api/v1/cities?prefCode={}"

# 環境変数からAPIキーを取得
api_key = os.getenv("RESAS_API_KEY")

headers = {
    "X-API-KEY": api_key
}

response = requests.get(pref_url , headers=headers)
prefectures = response.json()['result']

# pref_codeとpref_nameの辞書を作成
pref_code_to_name = {str(pref['prefCode']).zfill(2): pref['prefName'] for pref in prefectures}

# 全市区町村情報を取得
city_code_to_name = {}
for pref in prefectures:
    pref_code = str(pref['prefCode'])
    response = requests.get(city_url.format(pref_code), headers=headers)
    cities = response.json()['result']
    for city in cities:
        city_code_to_name[city['cityCode']] = city['cityName']

pref_code_to_name
city_code_to_name

{'01100': '札幌市',
 '01101': '札幌市中央区',
 '01102': '札幌市北区',
 '01103': '札幌市東区',
 '01104': '札幌市白石区',
 '01105': '札幌市豊平区',
 '01106': '札幌市南区',
 '01107': '札幌市西区',
 '01108': '札幌市厚別区',
 '01109': '札幌市手稲区',
 '01110': '札幌市清田区',
 '01202': '函館市',
 '01203': '小樽市',
 '01204': '旭川市',
 '01205': '室蘭市',
 '01206': '釧路市',
 '01207': '帯広市',
 '01208': '北見市',
 '01209': '夕張市',
 '01210': '岩見沢市',
 '01211': '網走市',
 '01212': '留萌市',
 '01213': '苫小牧市',
 '01214': '稚内市',
 '01215': '美唄市',
 '01216': '芦別市',
 '01217': '江別市',
 '01218': '赤平市',
 '01219': '紋別市',
 '01220': '士別市',
 '01221': '名寄市',
 '01222': '三笠市',
 '01223': '根室市',
 '01224': '千歳市',
 '01225': '滝川市',
 '01226': '砂川市',
 '01227': '歌志内市',
 '01228': '深川市',
 '01229': '富良野市',
 '01230': '登別市',
 '01231': '恵庭市',
 '01233': '伊達市',
 '01234': '北広島市',
 '01235': '石狩市',
 '01236': '北斗市',
 '01303': '当別町',
 '01304': '新篠津村',
 '01331': '松前町',
 '01332': '福島町',
 '01333': '知内町',
 '01334': '木古内町',
 '01337': '七飯町',
 '01343': '鹿部町',
 '01345': '森町',
 '01346': '八雲町',
 '01347': '長万部町',
 '01361': '江差町'

In [27]:
# 県コードを抽出
extracted_df.loc[:, 'pref_code'] = extracted_df.loc[:, 'area_code'].str[:2]

# 年度ごとの県別平均価格を計算
pref_avg = extracted_df.groupby(['pref_code', 'year'])['price'].mean().reset_index()

print(pref_avg.head())
# 年次リターンの計算
pref_avg['return'] = pref_avg.groupby('pref_code')['price'].pct_change()

# 先頭行のリターンは計算できないため削除
pref_avg = pref_avg.dropna(subset=['return'])

pref_avg


  pref_code  year         price
0        01  2007  52941.663455
1        01  2008  56655.098684
2        01  2009  53923.840970
3        01  2010  50241.718963
4        01  2011  48500.160467


,pref_code,year,price,return
1,01,2008,56655.098684,0.070142
2,01,2009,53923.840970,-0.048209
3,01,2010,50241.718963,-0.068284
4,01,2011,48500.160467,-0.034664
5,01,2012,46950.014588,-0.031962
...,...,...,...,...
841,47,2020,136818.177083,0.147650
842,47,2021,137272.916667,0.003324
843,47,2022,138921.406250,0.012009
844,47,2023,143021.770833,0.029516


In [30]:
# 県別の指標を計算
pref_metrics = pref_avg.groupby('pref_code', group_keys=False).apply(calculate_metrics, include_groups=False)

# フォーマットされた結果を表示
formatted_pref_metrics = pref_metrics.apply(format_metrics, axis=1)

# 新しいカラムとして都道府県名を追加
formatted_pref_metrics['Prefecture Name'] = formatted_pref_metrics.index.map(pref_code_to_name.get)

formatted_pref_metrics.to_csv("pref_metrics.csv", encoding="utf-8-sig", index=True)
print("Prefecture Metrics:")
formatted_pref_metrics

Prefecture Metrics:


,CAGR,Geometric Mean Return,Annualized Volatility,Sharpe Ratio,Prefecture Name
pref_code,,,,,
01,2.81%,3.05%,5.45%,0.561,北海道
02,-2.15%,-2.39%,2.60%,-0.922,青森県
03,-2.05%,-2.21%,3.33%,-0.662,岩手県
04,3.08%,3.78%,6.45%,0.586,宮城県
05,-2.76%,-2.87%,3.27%,-0.878,秋田県
06,-1.54%,-1.70%,2.75%,-0.619,山形県
07,0.03%,-0.18%,2.90%,-0.063,福島県
08,-1.11%,-1.18%,2.34%,-0.503,茨城県
09,-1.38%,-1.41%,1.86%,-0.756,栃木県


In [31]:
# 市区町村コードを抽出
extracted_df.loc[:, 'city_code'] = extracted_df.loc[:, 'area_code'].str[:5]

# 年度ごとの市区町村別平均価格を計算
city_avg = extracted_df.groupby(['city_code', 'year'])['price'].mean().reset_index()

print(city_avg.head())
# 年次リターンの計算
city_avg['return'] = city_avg.groupby('city_code')['price'].pct_change()

# 先頭行のリターンは計算できないため削除
city_avg = city_avg.dropna(subset=['return'])

city_avg

  city_code  year          price
0     01101  2007  285590.909091
1     01101  2008  350130.769231
2     01101  2009  320781.250000
3     01101  2010  283218.750000
4     01101  2011  274075.000000


,city_code,year,price,return
1,01101,2008,350130.769231,0.225987
2,01101,2009,320781.250000,-0.083824
3,01101,2010,283218.750000,-0.117097
4,01101,2011,274075.000000,-0.032285
5,01101,2012,265241.666667,-0.032230
...,...,...,...,...
27764,47362,2020,67300.000000,0.078094
27765,47362,2021,67775.000000,0.007058
27766,47362,2022,68775.000000,0.014755
27767,47362,2023,71600.000000,0.041076


In [32]:
# 県別の指標を計算
city_metrics = city_avg.groupby('city_code', group_keys=False).apply(calculate_metrics, include_groups=False)

# フォーマットされた結果を表示
formatted_city_metrics = city_metrics.apply(format_metrics, axis=1)

# 新しいカラムとして市区町村名を追加
formatted_city_metrics['City Name'] = formatted_city_metrics.index.map(city_code_to_name.get)

formatted_city_metrics.to_csv("city_metrics.csv", encoding="utf-8-sig", index=True)
print("City Metrics:")
formatted_city_metrics

City Metrics:


,CAGR,Geometric Mean Return,Annualized Volatility,Sharpe Ratio,City Name
city_code,,,,,
01101,4.71%,5.68%,9.39%,0.605,札幌市中央区
01102,4.83%,5.14%,6.33%,0.813,札幌市北区
01103,3.23%,3.40%,5.92%,0.573,札幌市東区
01104,3.02%,3.15%,5.35%,0.590,札幌市白石区
01105,2.91%,3.20%,5.34%,0.599,札幌市豊平区
...,...,...,...,...,...
47328,8.20%,7.48%,37.92%,0.197,中城村
47329,2.54%,2.27%,3.38%,0.671,西原町
47348,1.71%,1.42%,4.17%,0.339,与那原町
